In [1]:
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import pandas as pd
import cartopy.crs as ccrs
from metpy.units import units
from tqdm import tqdm
from src.utils import compute_A

In [2]:
# Combine all files in path 
# select var 
var = 'tas'
sim_number = 0
path = f"/glade/derecho/scratch/awikner/PLASIM/data/2000_year_sims_new/sim{sim_number}/{var}/"
# select lat, lon and window size
spatial_window_size = 2
temporal_window_size = 7
# Select lat and lon of Chicago
lat = 41.881832
lon = -87.623177 % 360 # for longitudes take the value modulo 360


files = [f for f in os.listdir(path) if f.endswith('gaussian.nc')]
files = files[:10]

# disable the printing of the warning
# import warnings
# warnings.filterwarnings('ignore')

# # combine all files using compute_A as a preprocessing function 
A = xr.open_mfdataset([path+file for file in files], preprocess=lambda ds: compute_A(ds, lat, lon, spatial_window_size, temporal_window_size), combine='nested',
                       concat_dim='time', parallel=True, decode_times=True, use_cftime=True)

# # combine all files using the compute_A as a preprocessing step #, add tqdm to see the progress
# A = xr.concat([compute_A(xr.open_dataset(path + f), lat, lon, spatial_window_size, temporal_window_size) for f in tqdm(files)], dim='time')
# A.to_netcdf(f'data/A_{var}_{int(lat)}_{int(lon)}_{spatial_window_size}_{temporal_window_size}.nc')

A_df = A.to_dataframe()
# save the dataframe
A_df.to_csv(f'data/A_{var}_{int(lat)}_{int(lon)}_{spatial_window_size}_{temporal_window_size}.csv')

In [68]:
# # convert index to pandas.DatetimeIndex
# A_df = A.to_dataframe()

# # select month 6, 7 or 8
# A_df = A_df[np.isin(A_df.index.month, [6, 7, 8])]
# A_df

In [67]:
path = "/glade/work/alancelin/EVT_rossbypalooza/data/A_tas_41_272_2_7.csv"
A_df = pd.read_csv(path, index_col=0)
print(f"Memory usage of A_df: {A_df.memory_usage().sum() / 1024**2} MB")

Memory usage of A_df: 0.051605224609375 MB
